In [ ]:
import itertools

In [ ]:
import h5py
import mpmath
from mpmath import mp

In [ ]:
eye3 = mp.eye(3)

def spheroid_mp(a, c, n):
    a2 = a * a
    c2_minus_a2 = c * c - a2
    return (c*c-a2)*n*n.T+a2*eye3

In [ ]:
def f_mp(lambda_, r12, q1, q2):
    one_minus_lambda = 1-lambda_
    q = one_minus_lambda*q1+lambda_*q2
    s12, _ = mp.qr_solve(q, r12)
    return lambda_*one_minus_lambda*(r12.T*s12)[0, 0]

In [ ]:
radii = [mp.mpf(1999)/mp.mpf(100000),
         mp.mpf(1999)/mp.mpf(1000),
         mp.mpf(99999)/mp.mpf(1000)]

In [ ]:
golden_ratio = (mp.mpf(1)+mp.sqrt(mp.mpf(5)))/(mp.mpf(2))
u_abs = 1./mp.sqrt(1+golden_ratio**2)
v_abs = golden_ratio*u_abs
directions = []
for u in (-u_abs, u_abs):
    for v in (-v_abs, v_abs):
        directions += [mp.matrix((0., u, v)),
                       mp.matrix((v, 0., u)),
                       mp.matrix((u, v, 0.))]

In [ ]:
spheroids = [spheroid_mp(ai, ci, ni) for ai, ci, ni in itertools.product(radii, radii, directions)]

In [ ]:
num_lambdas = 11
lambdas = mp.linspace(0, 1, num_lambdas)

In [ ]:
num_radii = len(radii)
num_dirs = len(directions)
num_spheroids = len(spheroids)
num_blocks = num_spheroids**2

with h5py.File('test.h5', 'w') as f:
    f['radii'] = [float(r) for r in radii]
    f['directions'] = [(float(x), float(y), float(z)) for x, y, z in directions]
    f['lambdas'] = [float(val) for val in lambdas]
    dset = f.create_dataset('spheroids', shape=(num_spheroids, 6), dtype='d')
    for i, q in enumerate(spheroids):
        dset[i, 0] = float(q[0, 0])
        dset[i, 1] = float(q[0, 1])
        dset[i, 2] = float(q[0, 2])
        dset[i, 3] = float(q[1, 1])
        dset[i, 4] = float(q[1, 2])
        dset[i, 5] = float(q[2, 2])
    
    shape = (num_spheroids, num_spheroids,
             num_dirs, num_lambdas)
    
    dset = f.create_dataset('f', shape=shape, dtype='d')
    block = np.empty((num_dirs, num_lambdas), dtype=np.float64)
    for i1, q1 in enumerate(spheroids):
        print(f'{i1+1}/{num_spheroids}')
        for i2, q2 in enumerate(spheroids):
            for i, r12_dir in enumerate(directions):
                for j, lambda_ in enumerate(lambdas):
                    block[i, j] = float(f_mp(lambda_, r12_dir, q1, q2))
            dset[i1, i2, :, :] = block[:, :]
            f.flush()